# LSTM part-of-speech tagging for the French Treebank: 

This notebook trains a part-of-speech tagger for the French Treebank using a vanilla bi-direction LSTM network.

Run the following cell to load the Keras packages.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

from keras.models import Model
from keras.layers import Bidirectional, Dense, Input, Dropout, LSTM, Activation, TimeDistributed, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.initializers import glorot_uniform
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

%matplotlib inline

np.random.seed(1)

Using TensorFlow backend.


In [3]:
# normalizes word according to the conventions of fastText; only transforms the word to lower case

def normalize_word(orig_word):
    word = orig_word.lower()
    return word
    

In [4]:
def read_maxentdata(file):
    with open(file, 'r') as f:
        vocabulary = set()
        vnorm = set()
        partsofspeech1 = set()
        partsofspeech2 = set()
        superset = set()
        sentno = 0
        maxlen = 0
        words = []
        postags1 = []
        postags2 = []
        supertags = []
        allwords = []
        allpos1 = []
        allpos2 = []
        allsuper = []
        for line in f:
            line = line.strip().split()
            length = len(line)
            if (length > maxlen):
                maxlen = length
            for l in range(length):
                item = line[l].split('|')
                orig_word = item[0]
                word = normalize_word(orig_word)
                postag = item[1]
                supertag = item[2]
                poslist = postag.split('-')
                pos1 = poslist[0]
                pos2 = poslist[1]
                vocabulary.add(orig_word)
                vnorm.add(word)
                partsofspeech1.add(pos1)
                partsofspeech2.add(pos2)
                superset.add(supertag)
                words.append(orig_word)
                postags1.append(pos1)
                postags2.append(pos2)
                supertags.append(supertag)
            allwords.append(words)
            allpos1.append(postags1)
            allpos2.append(postags2)
            allsuper.append(supertags)
            words = []
            postags1 = []
            postags2 = []
            supertags = []
            
        X = np.asarray(allwords)
        Y1 = np.asarray(allpos1)
        Y2 = np.asarray(allpos2)
        Z = np.asarray(allsuper)
        return X, Y1, Y2, Z, vocabulary, vnorm, partsofspeech1, partsofspeech2, superset, maxlen

In [5]:
X, Y1, Y2, Z, vocabulary, vnorm, partsofspeech1, partsofspeech2, superset, maxLen = read_maxentdata('m2.txt')
# X, Y1, Y2, Z, vocabulary, partsofspeech1, partsofspeech2, superset, maxLen = read_maxentdata('aa1.txt')

print(X[1])
print(Y2[1])

numClasses = len(partsofspeech2)+1
numSuperClasses = len(superset)+1

print()
print("Longest sentence   : ", maxLen)
print("Number of POS tags : ", numClasses)
print("Number of supertags: ", numSuperClasses)


['La', 'Caixa', 'prend', '49', '%', 'de', 'Lyonnaise', 'Espana', ',', 'dont', 'le', 'groupe', 'français', 'ne', 'détiendra', 'plus', 'que', '51', '%', ';', 'en', 'échange', ',', 'elle', 'cède', 'à', 'celui', '-ci', '3,5', '%', 'de', 'la', 'SGAB', 'et', '19,6', '%', "d'", 'ACESA', ',', 'première', 'société', 'concessionnaire', "d'", 'autoroutes', 'en', 'Espagne', '(', '500', 'kilomètres', "d'", 'autoroutes', 'et', 'un', 'chiffre', "d'", 'affaires', '1990', 'de', '37', 'milliards', 'de', 'pesetas', ')', ',', 'dont', 'elle', 'conserve', '26,6', '%', '.']
['DET:ART', 'NAM', 'VER:pres', 'NUM', 'SYM', 'PRP', 'NAM', 'NAM', 'PUN', 'PRO:REL', 'DET:ART', 'NOM', 'ADJ', 'ADV', 'VER:futu', 'ADV', 'ADV', 'NUM', 'SYM', 'PUN', 'PRP', 'NOM', 'PUN', 'PRO:PER', 'VER:pres', 'PRP', 'PRO:DEM', 'ADV', 'NUM', 'SYM', 'PRP', 'DET:ART', 'ABR', 'KON', 'NUM', 'SYM', 'PRP', 'ABR', 'PUN', 'NUM', 'NOM', 'ADJ', 'PRP', 'NOM', 'PRP', 'NAM', 'PUN', 'NUM', 'NOM', 'PRP', 'NOM', 'KON', 'DET:ART', 'NOM', 'PRP', 'NOM', 'NUM',

## 1. Split the input into train/dev/test

Split the full training set into 60% train, 20% dev and 20% test.

In [6]:
# split the training data into the standard 60% train, 20% dev, 20% test 
X_train, X_testdev, Y_train, Y_testdev = train_test_split(X, Y2, test_size=0.4)
X_test, X_dev, Y_test, Y_dev = train_test_split(X_testdev, Y_testdev, test_size=0.5)
print("Train: ", X_train.shape)
print("Test:  ", X_test.shape)
print("Dev:   ", X_dev.shape)


Train:  (9449,)
Test:   (3150,)
Dev:    (3150,)


## 2. Create auxiliary mappings

Create mappings from supertags and the two sets of part-of-speech tags to integers and back.

In [7]:
# create a bi-directional mapping (using two dictionaries) translating elements of a set to and from integers

def indexify (set):
    i = 1
    item_to_index = {}
    index_to_item = {}

    for item in set:
        item_to_index[item] = i
        index_to_item[i] = item
        i = i + 1

    return item_to_index, index_to_item    

In [8]:
# create mapping for the two POS tagset and for the supertags

super_to_index, index_to_super = indexify(superset)
pos1_to_index, index_to_pos1 = indexify(partsofspeech1)
pos2_to_index, index_to_pos2 = indexify(partsofspeech2)
print(pos2_to_index)

{'PUN:cit': 1, 'VER:futu': 2, 'DET:ART': 3, 'PRO:POS': 21, 'DET:POS': 24, 'NAM': 4, 'VER:simp': 5, 'PRO:REL': 23, 'PRP': 6, 'VER:pres': 7, 'VER:cond': 8, 'PRO': 25, 'SYM': 9, 'INT': 11, 'VER:infi': 12, 'PRP:det': 26, 'VER:impe': 27, 'NOM': 13, 'VER:subp': 14, 'VER:pper': 15, 'VER:impf': 16, 'ADJ': 28, 'PRO:PER': 29, 'PRO:IND': 17, 'PUN': 18, 'VER:ppre': 30, 'ADV': 31, 'ABR': 22, 'NUM': 19, 'KON': 10, 'PRO:DEM': 20}


## 3. Obtain the word vector information

We are using a shell call to the compiled fastText code to produce a file _vectors.txt_ with the relevant vectors. 

### 3.1. Special treatment for numbers

fastText does not include native numbers; internally, these are translated into sequences of the words "zéro", "un", etc. 

We add new entries averaging over the number symbols for numbers appearing in the French Treebank.

In [9]:
with open("num_vec.txt", 'r') as f:
    number = {}
    for line in f:
        line = line.strip().split()
        numc = line[0]
        emb = np.array(line[1:], dtype=np.float64)
        number[numc] = emb

In [10]:
for i in number.keys():
    print(i)

1
9
3
5
7
,
4
8
0
.
6
2
/
-


In [11]:
def is_numeral (string):
    return re.match(r'\A\-?\d[\d\.\,-/]*\Z', string) is not None

In [12]:
print(is_numeral("-1.234,56"))
print(is_numeral("1 2 3 "))
print(is_numeral("3/4"))
print(is_numeral("+1"))

True
False
True
False


In [13]:
def num_to_vec (ns):
    vecsize = np.size(number["0"])
    avg = np.zeros(vecsize)

    cl = list(ns)
    i = 0
    for char in cl:
        avg += number[char]
        i += 1
    avg = avg/i
    return avg

In [14]:
num_to_vec("1/4")

array([ -1.63150000e-01,   1.27537333e-01,  -1.14730100e-01,
         2.43636667e-01,   1.12086667e-01,  -1.32714333e-01,
         2.47716667e-02,   6.04950000e-02,  -1.29726667e-02,
         1.87775000e-01,  -1.68472333e-01,   8.62280000e-02,
        -1.88148667e-01,   1.57618333e-01,  -2.10669667e-01,
         5.37993333e-02,  -4.35270000e-02,   5.43833333e-02,
        -7.87723333e-02,  -1.28075333e-01,   1.40230333e-01,
        -1.07276667e-01,  -3.42530000e-02,  -9.27800000e-03,
        -2.79923333e-02,  -9.28250000e-02,   2.37066667e-03,
        -1.11023333e-01,  -2.42835667e-01,  -9.00413333e-02,
         5.75456667e-02,   1.85236667e-01,   2.68430000e-02,
         1.75517667e-01,   1.87653000e-01,   1.98363333e-02,
         1.10892267e-01,  -6.92060000e-02,   2.81330000e-02,
         1.92766333e-01,   5.47740000e-02,   2.61433333e-03,
         1.49333333e-02,  -8.27696667e-02,   3.63566667e-02,
        -1.41419333e-01,  -4.82583333e-02,   7.40089600e-02,
        -9.76400000e-02,

### 3.2. Feature vectors

#### Suffixes

In [15]:
def read_suffixes(file):
    i = 1
    suffixes = {}

    with open(file, 'r') as f:
        for line in f:
            line = line.strip()
            suffixes[line] = i
            i = i + 1

    return suffixes       

In [17]:
french_suffixes = read_suffixes('suffixes.txt')
print(french_suffixes)
print(len(french_suffixes))

{'thermique': 191, 'phobie': 178, 'unième': 197, 'èdre': 61, 'esque': 83, 'erais': 75, 'at': 40, 'âtre': 47, 'ification': 113, 'ier': 108, 'in': 115, 'ible': 101, 'ie': 103, 'génie': 96, 'eront': 82, 'manie': 142, 'o': 148, 'adique': 8, 'ment': 143, 'morphe': 146, 'oir': 153, 'aine': 17, 'eur': 88, 'euse': 89, 'assions': 39, 'ai': 10, 'oient': 151, 'eraient': 74, 'eraie': 73, 'issime': 126, 'vore': 200, 'lingue': 133, 'phore': 180, 'ance': 27, 'aient': 12, 'ât': 41, 'erai': 72, 'graphie': 100, 'erie': 80, 'ité': 131, 'ée': 63, 'ine': 116, 'scope': 187, 'oyer': 171, 'er': 71, 'aison': 22, 'ème': 66, 's': 186, 'graphe': 99, 'ole': 158, 'amment': 26, 'itude': 132, 'et': 85, 'ienne': 107, 'ure': 198, 'elle': 65, 'assent': 36, 'muche': 147, 'ît': 129, 'able': 2, 'ificateur': 112, 'oise': 156, 'teur': 190, 'ane': 28, 'émie': 67, 'esse': 84, 'iez': 110, 'gène': 95, 'ons': 161, 'os': 162, 'ptile': 183, 'ate': 42, 'assiez': 38, 'erez': 79, 'trice': 194, 're': 185, 'e': 56, 'oche': 149, 'aud': 4

In [18]:
def suffix_vector(word, suffixes=french_suffixes):
    length = len(suffixes)+1
    vector = np.zeros(length)
    for suf,num in suffixes.items():
        if word.endswith(suf):
            vector[num] = 1.0
        else:
            vector[num] = 0.0
    return vector        

In [20]:
suffix_vector("seraient", french_suffixes)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

#### Manually designed features

In [21]:
def word_features(word, unknown=False):
    list = []
    if word.isupper():
        list.append(1.0)
    else:
        list.append(0.0)
    if word[0].isupper():
        list.append(1.0)
    else:
        list.append(0.0)
    if word.isnumeric():
        list.append(1.0)
    else:
        list.append(0.0)
    if word.isalnum():
        list.append(0.0)
    else:
        list.append(1.0)
    if word[0] == "-":
         list.append(1.0)
    else:
        list.append(0.0)
    if is_numeral(word):
         list.append(1.0)
    else:
        list.append(0.0)        
    if "-" in word[1:]:
        list.append(1.0)
    else:
        list.append(0.0)
    if unknown:
         list.append(1.0)
    else:
        list.append(0.0)
    
    ar = np.asarray(list)
    return ar

In [22]:
print(word_features("ABCD"))
print(word_features("Abcd"))
print(word_features("1234"))
print(word_features("*%"))
print(word_features("Ab-cd"))
print(word_features("-t-il"))


[ 1.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  1.  0.  0.]
[ 0.  0.  0.  1.  0.  0.  0.  0.]
[ 0.  1.  0.  1.  0.  0.  1.  0.]
[ 0.  0.  0.  1.  1.  0.  1.  0.]


### 3.3. Sending the vocabulary through the fasttext executable

Write the vocabulary to an output file, then pass it to the fastText executable to produce the relevant word embeddings for our text. Since the fastText model is over 5 Gb, the shell call can take some time.

In [23]:
with open("vocab.txt", 'w') as vocab_file:
    for w in vnorm:
        print(w, file=vocab_file)

Shell call to `fasttext` for my Macbook Air

In [24]:
!/Users/moot/Software/fastText-master/fasttext print-word-vectors /Users/moot/Corpus/wiki.fr/wiki.fr.bin < vocab.txt > vectors.txt

Shell call to `fasttest` for my Macbook Pro, with `wiki.fr.bin` on external drive

In [ ]:
!/Users/moot/Software/fastText-master/fasttext print-word-vectors /Volumes/LaCie/Corpus/fastText/wiki/wiki.fr.bin < vocab.txt > vectors.txt

#### Combine all vector information

Combined all information from fasttext, suffixes, manually selected features and the features for digits (which are not in fasttext) to produce combined feature vectors for all words 

In [25]:
def read_vecs(file):
    with open(file, 'r') as f:
        words = set()
        vocabn = vnorm
        vocab = vocabulary
        word_to_vec_map = {}
        emsize = 0
        # special treatment for numerals
        numset = set()
        for w in vocabn:
            if is_numeral(w):
                numset.add(w)
                words.add(w)
                emb = num_to_vec(w)
                features = word_features(w)
                suf = suffix_vector(w)
                word_to_vec_map[w] = np.concatenate((emb,suf,features))
        vocabn = vocabn.difference(numset)
        vocab = vocab.difference(numset)
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            if (curr_word in vocabn):
                vocabn.remove(curr_word)
                vocab.discard(curr_word)
                words.add(curr_word)                
                emb = np.array(line[1:], dtype=np.float64)
                emsize = np.size(emb)
                features = word_features(curr_word)
                suf = suffix_vector(curr_word)
                word_to_vec_map[curr_word] = np.concatenate((emb,suf,features))

        for w in vocab:
            words.add(w)
            wn = normalize_word(w)
            emb = np.zeros(emsize)
            suf = suffix_vector(wn)
            try:
                vec = word_to_vec_map[wn]
            except:  
                print(w)
                features = word_features(w, unknown=True)
                word_to_vec_map[w] = np.concatenate((emb,suf,features))
            else:
                emb = vec[0:emsize]
                features = word_features(w)
                word_to_vec_map[w] = np.concatenate((emb,suf,features))

        for w in vocabn:
            words.add(w)
            try:
                vec = word_to_vec_map[w]
            except:  
                print(w)
                features = word_features(w, unknown=True)
                emb = np.zeros(emsize)
                suf = suffix_vector(w)
                word_to_vec_map[w] = np.concatenate((emb,suf,features))
        
                
        i = 2  # keep 1 for unknown
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


In [26]:
word_to_index, index_to_word, word_to_vec_map = read_vecs('vectors.txt')

### 3.4 - The Embedding layer

In Keras, the embedding matrix is represented as a "layer", and maps positive integers (indices corresponding to words) into dense vectors of fixed size (the embedding vectors). It can be trained or initialized with a pretrained embedding. In this part, we create an [Embedding()](https://keras.io/layers/embeddings/) layer in Keras, and initialize it with the fastTeX vectors loaded earlier in the notebook. 

The `Embedding()` layer takes an integer matrix of size (batch size, max input length) as input. This corresponds to sentences converted into lists of indices (integers), as shown in the figure below.

The largest integer (i.e. word index) in the input should be no larger than the vocabulary size. The layer outputs an array of shape (batch size, max input length, dimension of word vectors).

We first convert all our training sentences into lists of indices, and then zero-pad all these lists so that their length is the length of the longest sentence. 

**TODO**: I'd like try if it makes any difference to add the </s> end tag to the end of each sentence. 

In [27]:
def lists_to_indices(X, item_to_index, max_len, normalize=False):

    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))

    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split it into words. You should get a list of words.
        list = X[i]

        j = 0
        
        # Loop over the words of sentence_words
        for w in list:
            if normalize == True:
                w = normalize_word(w)
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
                print("Unknown: ", w)
                X_indices[i, j] = item_to_index[w]
            except:
                X_indices[i, j] = 1  # unknown
            # Increment j to j + 1
            j = j + 1
            
    return X_indices


In [28]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split it into words. You should get a list of words.
        sentence_words = X[i]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            w = normalize_word(w)
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
                X_indices[i, j] = word_to_index[w]
            except:
                print("Unknown: ", w)
                X_indices[i, j] = 1   # index for unknown words
            # Increment j to j + 1
            j = j + 1
            
    ### END CODE HERE ###
    
    return X_indices

Run the following cell to check what `sentences_to_indices()` does, and check your results.

In [29]:
sentences_to_indices(X_train, word_to_index, maxLen)

array([[  1.56960000e+04,   1.84410000e+04,   1.40000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.29150000e+04,   1.44590000e+04,   1.56960000e+04, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.54280000e+04,   1.45310000e+04,   1.75100000e+03, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  1.76880000e+04,   1.47850000e+04,   1.40000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.29150000e+04,   2.73950000e+04,   2.42880000e+04, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.22700000e+04,   9.78500000e+03,   3.34540000e+04, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

We now build the `Embedding()` layer for use with Keras, using pre-trained word vectors. After this layer is built, we can pass the output of `sentences_to_indices()` to it as an input, and the `Embedding()` layer will return the word embeddings for a sentence. 

We use the following steps:
1. Initialize the embedding matrix as a numpy array of zeroes with the correct shape.
2. Fill in the embedding matrix with all the word embeddings extracted from `word_to_vec_map`.
3. Define Keras embedding layer. Use [Embedding()](https://keras.io/layers/embeddings/). Be sure to make this layer non-trainable, by setting `trainable = False` when calling `Embedding()`. If you were to set `trainable = True`, then it will allow the optimization algorithm to modify the values of the word embeddings. 
4. Set the embedding weights to be equal to the embedding matrix 

In [30]:

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 2                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["aime"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len,emb_dim,trainable=False,mask_zero=True)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [40]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][2][1] =", embedding_layer.get_weights()[0][2][1])

weights[0][2][1] = 0.19175


**Expected Output**:

<table>
    <tr>
        <td>
            **weights[0][2][1] =**
        </td>
        <td>
           0.19175
        </td>
    </tr>
</table>

## 4. Building the Part-of-Speech tagger

We now build the POS-tagger model using the previously built enbedding layer and feed its output to a bidirectional LSTM network with 128 states in each direction. 



In [42]:
# POS_model

def POS_model(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # returning a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    X = BatchNormalization()(X)
    Y = Dropout(0.2)(X)
    # Add a (time distributed) Dense layer followed by a softmax activation
    Y = TimeDistributed(Dense(numClasses, activation='softmax'))(Y)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices,outputs=Y)
        
    return model

Run the following cell to create your model and check its summary. Because all sentences in the dataset are less than 10 words, we chose `max_len = 10`.  You should see your architecture, it uses "20,223,927" parameters, of which 20,000,050 (the word embeddings) are non-trainable, and the remaining 223,877 are. Because our vocabulary size has 400,001 words (with valid indices from 0 to 400,000) there are 400,001\*50 = 20,000,050 non-trainable parameters. 

In [43]:
model = POS_model((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 266)               0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 266, 512)          18222080  
_________________________________________________________________
lstm_1 (LSTM)                (None, 266, 128)          328192    
_________________________________________________________________
batch_normalization_1 (Batch (None, 266, 128)          512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 266, 128)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 266, 32)           4128      
Total params: 18,554,912
Trainable params: 332,576
Non-trainable params: 18,222,336
__________________________________________________________

As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics:

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

It's time to train your model. Your Emojifier-V2 `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`). We thus have to convert X_train (array of sentences as strings) to X_train_indices (array of sentences as list of word indices), and Y_train (labels as indices) to Y_train_oh (labels as one-hot vectors).

In [45]:
print(pos2_to_index)
X_train_indices = lists_to_indices(X_train, word_to_index, maxLen)
Y_train_indices = lists_to_indices(Y_train, pos2_to_index, maxLen)
Y_train_oh = to_categorical(Y_train_indices, num_classes=numClasses)

{'PUN:cit': 1, 'VER:futu': 2, 'DET:ART': 3, 'PRO:POS': 21, 'DET:POS': 24, 'NAM': 4, 'VER:simp': 5, 'PRO:REL': 23, 'PRP': 6, 'VER:pres': 7, 'VER:cond': 8, 'PRO': 25, 'SYM': 9, 'INT': 11, 'VER:infi': 12, 'PRP:det': 26, 'VER:impe': 27, 'NOM': 13, 'VER:subp': 14, 'VER:pper': 15, 'VER:impf': 16, 'ADJ': 28, 'PRO:PER': 29, 'PRO:IND': 17, 'PUN': 18, 'VER:ppre': 30, 'ADV': 31, 'ABR': 22, 'NUM': 19, 'KON': 10, 'PRO:DEM': 20}


In [46]:
print(Y_train_indices[1])
print(Y_train_oh[1])

[  3.   4.  26.   4.   6.  13.  18.  22.  18.  16.  15.   3.  19.  13.  26.
  28.   4.   4.   6.  12.   3.  13.  28.   6.   4.  18.   6.  12.  15.  10.
   3.  19.  13.  13.  28.  16.  15.  18.   6.  29.  12.  18.   3.  13.  26.
  13.  26.  13.   6.  20.  13.  18.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  

Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
9449/9449 [==============================] - 481s 51ms/step - loss: 0.4102 - acc: 0.8951
Epoch 2/50
9449/9449 [==============================] - 496s 52ms/step - loss: 0.0883 - acc: 0.9752
Epoch 3/50
9449/9449 [==============================] - 537s 57ms/step - loss: 0.0697 - acc: 0.9795
Epoch 4/50
9449/9449 [==============================] - 553s 58ms/step - loss: 0.0600 - acc: 0.9822
Epoch 5/50
9449/9449 [==============================] - 537s 57ms/step - loss: 0.0541 - acc: 0.9836
Epoch 6/50
9449/9449 [==============================] - 591s 63ms/step - loss: 0.0486 - acc: 0.9850
Epoch 7/50
9449/9449 [==============================] - 545s 58ms/step - loss: 0.0444 - acc: 0.9862
Epoch 8/50
9449/9449 [==============================] - 524s 55ms/step - loss: 0.0404 - acc: 0.9873
Epoch 9/50
9449/9449 [==============================] - 553s 59ms/step - loss: 0.0366 - acc: 0.9886
Epoch 10/50
9449/9449 [==============================] - 562s 59ms/step - loss: 0.0340 - acc: 0.9893

Your model should perform close to **100% accuracy** on the training set. The exact accuracy you get may be a little different. Run the following cell to evaluate your model on the test set. 

In [331]:
X_dev_indices = lists_to_indices(X_dev, word_to_index, max_len = maxLen)
Y_dev_indices = lists_to_indices(Y_dev, pos2_to_index, max_len = maxLen)
Y_dev_oh = to_categorical(Y_dev_indices, num_classes = numClasses)
loss, acc = model.evaluate(X_dev_indices, Y_dev_oh)
print()
print("Test accuracy = ", acc)

3150/3150 [==============================] - 20s 6ms/step

Test accuracy =  0.98499766505


You should get a test accuracy of about 94.8% for a vanilla model using only aa1.txt.
A vanilla POS model on the full training set gets a dev accuracy of 98.50%!

In [39]:
def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
    
    df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    df_conf_norm = df_confusion / df_confusion.sum(axis=1)
    
    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)


In [332]:
X_dev_indices = lists_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_dev_indices)

# print('           '+ label_to_emoji(0)+ '    ' + label_to_emoji(1) + '    ' +  label_to_emoji(2)+ '    ' + label_to_emoji(3)+'   ' + label_to_emoji(4))
# print(pd.crosstab(Y_dev, pred_test.reshape(56,), rownames=['Actual'], colnames=['Predicted'], margins=True))
plot_confusion_matrix(Y_dev, pred)

NameError: name 'pd' is not defined

In [333]:
# This code allows you to see the mislabelled examples

y_dev_oh = to_categorical(Y_dev_indices, num_classes = numClasses)
X_dev_indices = lists_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_dev_indices)

for i in range(len(X_dev)):
    x = X_dev_indices
    for j in range(maxLen):
        num = np.argmax(pred[i][j])
        if(num != Y_dev_indices[i][j]):
            print('Expected POS tag: '+ X_dev[i][j] + '|' + Y_dev[i][j] + ' prediction: '+ X_dev[i][j] + '|' + index_to_pos2[num])

Expected POS tag: La|DET:ART prediction: La|PRP
Expected POS tag: n'|ADV prediction: n'|NOM
Expected POS tag: est|VER:pres prediction: est|KON
Expected POS tag: pas|ADV prediction: pas|ADJ
Expected POS tag: réglementée|VER:pper prediction: réglementée|PUN
Expected POS tag: comme|ADV prediction: comme|PRO:DEM
Expected POS tag: celles|PRO:DEM prediction: celles|NOM
Expected POS tag: des|PRP:det prediction: des|PRO:PER
Expected POS tag: guides|NOM prediction: guides|VER:pres
Expected POS tag: interprètes|NOM prediction: interprètes|PUN:cit
Expected POS tag: ou|KON prediction: ou|PRP
Expected POS tag: des|PRP:det prediction: des|DET:ART
Expected POS tag: de|PRP prediction: de|VER:ppre
Expected POS tag: voyage|NOM prediction: voyage|PRP
Expected POS tag: .|PUN prediction: .|NOM


IndexError: list index out of range

### POStagger results on development set

| tagset | LTSM units | batchnorm | dropout | epochs | results |
|:-----|---------:|:----------:|-------:|-----:|----------:|
| tt | 128 | no | 0 |  50 | 98.50 |


In [334]:
model.save('tt_pos.h5')

## 5. Training the Supertagger

We split the data as before, only using Z (supertags) instead of Y2 (treetagger POStag set) as the goal

In [335]:
# split the training data into the standard 60% train, 20% dev, 20% test 
X_train, X_testdev, Y_train, Y_testdev = train_test_split(X, Z, test_size=0.4)
X_test, X_dev, Y_test, Y_dev = train_test_split(X_testdev, Y_testdev, test_size=0.5)
print("Train: ", X_train.shape)
print("Test:  ", X_test.shape)
print("Dev:   ", X_dev.shape)

Train:  (9449,)
Test:   (3150,)
Dev:    (3150,)


In [349]:
# Super_model

def Super_model(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # returning a batch of sequences.
    X = LSTM(256, return_sequences=True)(embeddings) 
    X = BatchNormalization()(X)
    X = Dropout(0.2)(X)
    
    # Add a (time distributed) Dense layer followed by a softmax activation
    X = TimeDistributed(Dense(numSuperClasses, activation='softmax'))(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices,outputs=X)
        
    return model

In [350]:
supermodel = Super_model((maxLen,), word_to_vec_map, word_to_index)
supermodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 266)               0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 266, 512)          18221056  
_________________________________________________________________
lstm_11 (LSTM)               (None, 266, 256)          787456    
_________________________________________________________________
batch_normalization_2 (Batch (None, 266, 256)          1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 266, 256)          0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 266, 891)          228987    
Total params: 19,238,523
Trainable params: 1,016,955
Non-trainable params: 18,221,568
________________________________________________________

In [351]:
supermodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [352]:
X_train_indices = lists_to_indices(X_train, word_to_index, maxLen)
Y_train_indices = lists_to_indices(Y_train, super_to_index, maxLen)
Y_train_oh = to_categorical(Y_train_indices, num_classes=numSuperClasses)

In [ ]:
print(Y_train_indices[2])
print(Y_train_oh[2])

[ 100.   22.  843.  844.   22.  874.   22.  874.   22.   23.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.   

In [ ]:
supermodel.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
9449/9449 [==============================] - 715s 76ms/step - loss: 1.3650 - acc: 0.7006
Epoch 2/50
9449/9449 [==============================] - 650s 69ms/step - loss: 0.7712 - acc: 0.7871
Epoch 3/50
9449/9449 [==============================] - 663s 70ms/step - loss: 0.6856 - acc: 0.8024
Epoch 4/50
9449/9449 [==============================] - 647s 69ms/step - loss: 0.6354 - acc: 0.8122
Epoch 5/50
9449/9449 [==============================] - 643s 68ms/step - loss: 0.5958 - acc: 0.8217
Epoch 6/50
4384/9449 [============>.................] - ETA: 5:53 - loss: 0.5541 - acc: 0.8302

In [348]:
X_dev_indices = lists_to_indices(X_dev, word_to_index, max_len = maxLen)
Y_dev_indices = lists_to_indices(Y_dev, super_to_index, max_len = maxLen)
Y_dev_oh = to_categorical(Y_dev_indices, num_classes = numSuperClasses)
loss, acc = supermodel.evaluate(X_dev_indices, Y_dev_oh)
print()
print("Test accuracy = ", acc)

3150/3150 [==============================] - 50s 16ms/step

Test accuracy =  0.802067654133


In [ ]:
model.save('supertagger.h5')

### Supertagger results on development set

| LTSM units | batchnorm | dropout | epochs | results |
|---------:|:----------:|-------:|----:|----------:|
| 128 | yes | .2 |  50 | 80.21 |
| 256 | yes | .2 |  50 | |